In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import userdata
from huggingface_hub import login

HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

print("Hugging Face logged in successfully")

In [ ]:
!pip install -q torch transformers datasets peft accelerate bitsandbytes trl faiss-cpu sentence-transformers
!pip install -U bitsandbytes

In [ ]:
import os

### Here show a sample of the testing dataset, which was built with the origin data from the open-source documents of LUMA MANTA project by ZJU-China 2025
### Test only, do not use this dataset for training or other unauthorized purposes. All rights reserved.

lora_dataset_content ="""
{"text": "[INST] Context: This experimental design document describes a molecular detection experiment based on the TetO/TetR system. Question: What is the goal of the first phase (PHASE-1) of the experiment? [/INST] The goal of PHASE-1 is to validate the TetO/TetR experiment using the PSR (Polymerase-based Strand displacement Reaction) version, adding signal gate and fuel gate to observe the final fluorescence signal."}
{"text": "[INST] Context: In PHASE-1, specific DNA templates were used. Question: What is the original sequence of the TetR_TetO_RecycleR_Template DNA? [/INST] The original TetR_TetO_RecycleR_Template sequence is: Sense strand: TAATACGACTCACTATAGGGTCCCTATCAGTGATAGAGAGTATCCAGTGAAGGAGTTGAACAGTGC, Antisense strand: GCACTGTTCААСТССТТСАCTGGATACTCTCTATCACTGATAGGGACCCTATAGTGAGTCGTATTA."}
{"text": "[INST] Context: The experiment modified the original DNA template. Question: What is the sequence of the revised TetR_TetO_RecycleR_Template DNA? [/INST] The revised DNA template sequence is: Sense strand: TAATACGACTCACTATAGGTCCCTATCAGTGATAGAGACGATCCAAATCCTCTCCGATCCAAATCCTCTCGAGAGGATTTGGATCGGAGAGGATTTGGATCGTCTCTATCACTGATAGGGACCTATAGTGAGTCGTATTA."}
{"text": "[INST] Context: The experiment used a Fuel Gate. Question: What are the two strands of the original Fuel Gate and their sequences? [/INST] The original Fuel Gate consists of Top Strand: ATCCAGTGAAGGAGTTGAACAGTGC; Bottom Strand: GCACTGTTCААСТССТТСACTGGAT."}
{"text": "[INST] Context: The Fuel Gate was also revised. Question: What are the sequences of the revised Fuel Gate (including Recycle D and Fuel Bottom Strand)? [/INST] In the revised Fuel Gate, Recycle D: AGATACTGTTCCCGATCCAA; Fuel Bottom Strand: GAGAGGATTTGGATCGGGAACAG."}
{"text": "[INST] Context: To prevent nonspecific reactions, certain DNA strands were chemically modified. Question: What chemical modifications were added to the Fuel Gate Bottom Strand and Quencher Strand? [/INST] To prevent nonspecific reactions and increase thermal stability, both the Fuel Gate Bottom Strand and Quencher Strand were fully modified with 2'-O-Methyl."}
{"text": "[INST] Context: The experiment generated fluorescence signals via Signal Gate. Question: What are the sequences of the FAM Strand (with fluorophore) and Quencher Strand in the original Signal Gate? [/INST] In the original Signal Gate, FAM Strand: /6-FAM/AGTACGACGAAGTCAAGAGTGAAGTTTGTTTTTT; Quencher Strand: AACTTCACTCTTGACTTCGTCGTACT/BHQ1/."}
{"text": "[INST] Context: The Quencher Strand in Signal Gate uses BHQ1 modification. Question: Why use BHQ1 instead of IowaBlackFQ? [/INST] Because only IDT in the USA can provide IowaBlackFQ modification, BHQ1 was used instead."}
{"text": "[INST] Context: Signal Gate design includes a Toehold. Question: What is the added Toehold sequence in Signal Gate and its function? [/INST] Signal Gate includes a 4-nucleotide 'TACT' Toehold, which amplifies strand displacement."}
{"text": "[INST] Context: There is also a revised version of Signal Gate. Question: What are the sequences of the FAM Strand and Quencher Strand in the revised Signal Gate? [/INST] In the revised Signal Gate, FAM Strand: TTGGATCGGGAACAGTATCTCGAG/FAM/; Quencher Strand: /BHQ1/mCmUmCmGAGATACTGmUmUmCmC."}
{"text": "[INST] Context: PSR reaction requires specific buffer. Question: What are the buffer components and final concentrations in a 20µL PSR reaction? [/INST] Buffer components and concentrations: Tris-HCl (40 mM, pH 8.0), NaCl (20 mM), MgCl₂ (8 mM), DTT (10 mM), Spermidine (2 mM)."}
{"text": "[INST] Context: TetR repressor protein was used. Question: What is the amino acid sequence of TetR protein? [/INST] TetR protein sequence: MSRLDKSKVINSALELLNEVGIEGLTTRKLAQKLGVEQPTLYWHVKNKRALLDALAIEMLDRHHTHFCPLEGESWQDFLRNNAKSFRCALLSHRDGAKVHLGTRPTEKQYETLENQLAFLCQQGFSLENALYALSAVGHFTLGCVLEDQEHQVAKEERETPTTDSMPPLLRQAIELFDHQGAEPAFLFGLELIICGLEKQLKCESGS."}
{"text": "[INST] Context: The annealing process of DNA templates is described. Question: What are the steps and conditions for DNA template annealing? [/INST] Mix equimolar sense and antisense strands in annealing buffer (10mM Tris-HCl, 50 mM NaCl, 1 mM EDTA), heat to 95°C for 5 min, then cool to 25°C at 1°C per 30 sec."}
{"text": "[INST] Context: Detailed PSR reaction components are described. Question: What are the final concentrations and volumes of each component in a 20µL PSR reaction? [/INST] In 20µL PSR: DNA template (1 nM, 0.2 µL), TetR protein (25 nM, 0.2 µL), Fuel gate (1.5 µM, 1.5 µL), Signal gate (2.5 µM, 2.5 µL), T7 RNA polymerase (2 ng, 0.2 µL), Tetracycline (1 µM, 0.2 µL), buffer components, and RNase-free water to 20µL."}
{"text": "[INST] Context: The second phase (PHASE-2) involves DNA immobilization. Question: What is the goal and principle of PHASE-2? [/INST] PHASE-2 aims to immobilize DNA fragments using the high affinity between biotin and streptavidin."}
{"text": "[INST] Context: DNA templates were modified for immobilization in PHASE-2. Question: What modification was made to the sense strand and its sequence? [/INST] The sense strand was modified with 5' Biotin: /5Biotin/TAATACGACTCACTATAGGGTCCCTATCAGTGATAGAGAGTATCCAGTGAAGGAGTTGAACAGTGC."}
{"text": "[INST] Context: The third phase (PHASE-3) aims to obtain fitting data. Question: What is the goal of PHASE-3 and how is it achieved? [/INST] PHASE-3 aims to obtain fitting data between tetracycline concentration and fluorescence signal by setting a gradient (0 to 5.0 µM, 9 steps) and measuring fluorescence over time."}
{"text": "[INST] Context: How is response intensity defined in PHASE-3 data analysis? Question: How to extract response intensity from fluorescence kinetics curves? [/INST] Response intensity can be extracted as the slope in the linear segment (0-30 min) or the endpoint value at 60 min."}
{"text": "[INST] Context: The fourth phase (PHASE-4) validates the SmtB/SmtO system. Question: What is the DNA template sequence used in PHASE-4? [/INST] PHASE-4 template (T7_smtO_RecycleR_Template): TAATACGACTCACTATAGGGAAGGCGGAGGTTGCGTATACAGGGTTATCCAGTGAAGGAGTTGAACAGTGC; reverse complement: GCACTGTTCAACTCCTTCACTGGATAACCCTGTATACGCAACCTCCGCCTTCCCTATAGTGAGTCGTATTA."}
{"text": "[INST] Context: The seventh phase (PHASE-7) involves logic gate design. Question: How to design OR and AND gates? [/INST] OR gate: merge different DNA templates; AND gate: split Fuel Gate Top Strand."}
{"text": "[INST] Context: The eighth phase (PHASE-8) proposes a cascade amplification reaction. Question: What is the principle of the two-layer PSR cascade amplification? [/INST] The first PSR loop produces RecycleD1, which activates the second PSR loop, releasing the second signal (FAM) for cascade amplification."}
{"text": "[INST] Context: A model prediction chart is provided. Question: According to the chart, what tetracycline concentration is needed for 50% DNA derepression at [TetR]/[DNA]=250/10? [/INST] At [TetR]/[DNA]=250/10 (nM/nM), about 0.1 µM tetracycline is needed for ~50% DNA derepression."}
{"text": "[INST] Context: A model prediction chart is provided. Question: At fixed tetracycline 10⁻³ µM, which [TetR]/[DNA] ratio gives the highest DNA derepression? [/INST] At tetracycline 10⁻³ µM, [TetR]/[DNA]=0.025/0.001 (nM/nM) gives the highest derepression, close to 1.0."}
{"text": "[INST] Context: The experiment designed an algae-bacteria symbiosis system, with cyanobacteria providing carbon. Question: What cyanobacteria strain was used and what is its engineering purpose? [/INST] Synechococcus elongatus UTEX 2973 was used, engineered for efficient sucrose secretion for E. coli carbon supply."}
{"text": "[INST] Context: Gene modification enables sucrose secretion in cyanobacteria. Question: Which key gene is overexpressed and under which promoter? [/INST] The cscB gene (sucrose permease) is overexpressed under the strong promoter P-cpc560."}
{"text": "[INST] Context: Sucrose secretion in cyanobacteria depends on induction conditions. Question: What conditions induce Synechococcus elongatus to secrete sucrose? [/INST] Sucrose secretion is induced by alkaline pH (8.3) and 150 mM NaCl."}
{"text": "[INST] Context: E. coli needs to be engineered to utilize sucrose. Question: Which genes are introduced into BL21(DE3) for sucrose utilization? [/INST] cscA (sucrose hydrolase), cscB (sucrose permease), cscK (fructokinase) are introduced; cscR (repressor) can be knocked out."}
{"text": "[INST] Context: Coculture conditions were optimized. Question: Why was the coculture temperature optimized from 37°C to 30°C? [/INST] At 37°C, E. coli consumes sucrose too fast (4.11 mg/L/h), causing imbalance. At 30°C, consumption slows to 2.00 mg/L/h, matching cyanobacteria secretion (4.20 mg/L/h) for stable coculture."}
{"text": "[INST] Context: Growth curve charts show coculture effects. Question: What is the average doubling rate (doublings/day) of the system? [/INST] Average coculture doubling rate is 0.71 ± 0.15 doublings/day."}
{"text": "[INST] Context: Plasmid for cscB integration in cyanobacteria was designed. Question: Which neutral site is used and what key elements are included? [/INST] The integration plasmid is based on NS3, includes 5'-NS3 and 3'-NS3 homology arms, promoter (Pcpc560), cscB gene, KanR, and lacI."}
{"text": "[INST] Context: Triparental conjugation steps are described. Question: Besides DH5α with target plasmid, which helper strain and plasmids are needed? [/INST] E. coli HB101 carrying pRL443 and pRL623 is used as helper strain."}
{"text": "[INST] Context: The SpyTag system assembles multi-enzyme complexes. Question: What is the principle of the SpyTag system? [/INST] A scaffold protein with multiple SpyCatchers assembles with functional enzymes carrying SpyTag via covalent bonds."}
{"text": "[INST] Context: Various peptides for heavy metal remediation are designed. Question: What enzyme is used for mercury (Hg) and its mechanism? [/INST] MerT/MerP (mercury reductase) reduces toxic Hg²⁺ to less volatile Hg⁰ for detoxification."}
{"text": "[INST] Context: α-amylase is proposed for heavy metal removal. Question: According to the chart, which metal does α-amylase remove most efficiently and what is the rate? [/INST] α-amylase removes cadmium (Cd) most efficiently, about 70% removal rate."}
{"text": "[INST] Context: Functional proteins for microplastic degradation are designed. Question: What is the fusion protein for polyester-polyurethane degradation and its components? [/INST] Tcur1278-TA2, composed of keratinase Tcur1278, 17x Helix, and anchor peptide Tachystatin A2."}
{"text": "[INST] Context: Light-controlled production layer uses red and blue light to control three proteins. Question: In the logic gate design, which aTF is expressed under red light and which is repressed? [/INST] Under red light, aTF1 is induced and aTF2 is repressed."}
{"text": "[INST] Context: Red and blue light signals trigger different small molecules. Question: Which three signal molecules are expressed after red light exposure? [/INST] IPTG, arabinose (ara), and tryptophan (Trp) are expressed after red light."}
{"text": "[INST] Context: What happens when both red and blue light are present in the logic gate? Question: Which aTF is expressed and how is self-stabilization achieved? [/INST] aTF3 is expressed; IPTG and aTC relieve lacI and tetR repression, CI1 inhibits repressor expression, keeping the pathway open."}
{"text": "[INST] Context: Plasmid map of pBR322-cscB is described. Question: What is the size and main elements of pBR322-cscB? [/INST] pBR322-cscB is 7159 bp, includes pBR322 backbone (ori, rop), KanR, cscB expression cassette (Pcpc560 promoter, TrbcL terminator), NS3 homology arms."}
{"text": "[INST] Context: E. coli growth under different sucrose concentrations is recorded. Question: At which sucrose concentration does E. coli reach highest OD600 after 48h and what is the value? [/INST] At 200 mg/L sucrose, OD600 reaches about 0.18-0.19 after 48h."}
{"text": "[INST] Context: In the detection layer, the original TetR_TetO_RecycleR_Template DNA adds GGG between T7 promoter and TetO. Question: What is the sense strand sequence of the original DNA template? [/INST] Sense strand: TAATACGACTCACTATAGGGTCCCTATCAGTGATAGAGAGTATCCAGTGAAGGAGTTGAACAGTGC."}
{"text": "[INST] Context: Fuel Gate Bottom Strand requires chemical modification for stability. Question: What modification is added to Fuel Gate Bottom Strand? [/INST] 2'-O-Methyl full modification is added for stability."}
{"text": "[INST] Context: Signal Gate Quencher Strand uses BHQ1. Question: Why not use IowaBlackFQ? [/INST] Only IDT in the USA can provide IowaBlackFQ, so BHQ1 is used."}
{"text": "[INST] Context: PSR buffer component concentrations are listed. Question: What is the final concentration and function of DTT in PSR? [/INST] DTT is 10mM, stabilizes enzyme structure."}
{"text": "[INST] Context: PSR buffer component concentrations are listed. Question: What is the final concentration and possible function of Spermidine in PSR? [/INST] Spermidine is 2mM, may promote transcription."}
{"text": "[INST] Context: TetR protein and its sequence are provided. Question: What is the full amino acid sequence of TetR protein? [/INST] TetR protein sequence: MSRLDKSKVINSALELLNEVGIEGLTTRKLAQKLGVEQPTLYWHVKNKRALLDALAIEMLDRHHTHFCPLEGESWQDFLRNNAKSFRCALLSHRDGAKVHLGTRPTEKQYETLENQLAFLCQQGFSLENALYALSAVGHFTLGCVLEDQEHQVAKEERETPTTDSMPPLLRQAIELFDHQGAEPAFLFGLELIICGLEKQLKCESGS."}
{"text": "[INST] Context: DNA template annealing buffer composition is described. Question: What are the three components of the annealing buffer? [/INST] 10mM Tris-HCl, 50 mM NaCl, 1 mM EDTA."}
{"text": "[INST] Context: Fuel Gate and Signal Gate annealing conditions are described. Question: After heating to 95°C, how long should Fuel Gate and Signal Gate be held before cooling? [/INST] Hold at 95°C for 5 min, then cool to 25°C."}
{"text": "[INST] Context: PSR reaction order is specified. Question: Which protein should DNA template be incubated with at room temperature for 10 min and why? [/INST] Incubate DNA template with TetR for 10 min to allow TetR binding to TetO."}
{"text": "[INST] Context: The last step of PSR is adding the inducer. Question: What is the last reagent added after all components? [/INST] Tetracycline is added last."}
{"text": "[INST] Context: TetR protein needs to be weighed and prepared. Question: How much dilution buffer is needed to prepare 20µg TetR (50kDa) at 2.5µM? [/INST] 160µL dilution buffer is needed for 20µg TetR at 2.5µM."}
{"text": "[INST] Context: DNA needs to be immobilized on streptavidin-coated slides in PHASE-2. Question: What modification is needed for DNA template? [/INST] Add 5' Biotin to the sense strand for immobilization."}
{"text": "[INST] Context: Preparation of streptavidin-coated slides is described. Question: What are the final concentrations of EDC and NHS? [/INST] EDC: 2mM; NHS: 5mM."}
{"text": "[INST] Context: Tetracycline gradient is set in PHASE-3. Question: What is the range of tetracycline concentrations for fitting data? [/INST] 0 µM to 5.0 µM, 9 gradients."}
{"text": "[INST] Context: SmtB/SmtO validation uses metal ions. Question: Which two ions are used in SmtB/SmtO validation? [/INST] Zinc (Zn) and copper (Cu) are used."}
{"text": "[INST] Context: AHL/eDNA detection is proposed in PHASE-6. Question: Which protein acts with AHL to initiate transcription? [/INST] LuxR acts with AHL to initiate transcription."}
{"text": "[INST] Context: AND gate design is described in PHASE-7. Question: Which component is split for AND gate design? [/INST] Fuel Gate Top Strand is split for AND gate."}
{"text": "[INST] Context: Model prediction minimum limit is mentioned. Question: What is the minimum effective TetR/DNA ratio according to the model? [/INST] 2.5nM/0.1nM TetR/DNA is the minimum effective ratio."}
{"text": "[INST] Context: Algae-bacteria symbiosis design is inspired by literature. Question: Which iGEM project inspired the design? [/INST] Inspired by 2021 iGEM Toulouse_INSA-UPS project."}
{"text": "[INST] Context: Cyanobacteria culture conditions are recorded. Question: According to the 2018 toolbox, which two antibiotics and concentrations are needed for stability? [/INST] Add 20 µg/mL chloramphenicol and/or 20 µg/mL spectinomycin for stability."}
{"text": "[INST] Context: Cyanobacteria culture conditions are recorded. Question: According to the 2021 CRISPR tool, what are the CO2 concentration and shaker speed? [/INST] CO2: 3%; shaker: 270 rpm."}
{"text": "[INST] Context: PCR template preparation from cyanobacteria is described. Question: How long to freeze and thaw for PCR template preparation? [/INST] Freeze at -80°C for 20 min, thaw at 60°C for 10 min, repeat once."}
{"text": "[INST] Context: Construction of high-sucrose cyanobacteria is described. Question: Which promoter regulates cscB gene expression? [/INST] IPTG-inducible lacUV5 promoter (P_lacUVs) regulates cscB."}
{"text": "[INST] Context: Triparental conjugation workflow is described. Question: What medium is used to wash E. coli to remove antibiotics? [/INST] Wash three times with fresh LB medium without antibiotics."}
{"text": "[INST] Context: Cyanobacteria freezing preservation is described. Question: What cryoprotectant and final concentration are recommended? [/INST] Use DMSO at 8% (v/v) for preservation."}
{"text": "[INST] Context: Sucrose quantification method is described. Question: At what pH does invertase hydrolyze sucrose? [/INST] Invertase hydrolyzes sucrose at pH 4.6."}
{"text": "[INST] Context: Sucrose quantification method is described. Question: At what wavelength is the red quinoneimine dye measured? [/INST] Measure absorbance at 510nm."}
{"text": "[INST] Context: Various E. coli genotypes are listed. Question: What is the genotype of DH5α? [/INST] DH5α genotype: F- endA1 glnV44 thi-1 recA1 relA1 gyrA96 deoR nupG purB20 Φ80dlacZΔM15 Δ(lacZYA-argF)U169, hsdR17(r- m+)."}
{"text": "[INST] Context: Various plasmids are used. Question: Which sucrose metabolism genes and resistance gene are on pET30a? [/INST] pET30a carries cscA, cscB, cscK, and kanR."}
{"text": "[INST] Context: May 16 work plan is recorded. Question: What type of primers are designed for BL21 cscBKA cluster? [/INST] Primers with restriction sites are designed for BL21 cscBKA cluster."}
{"text": "[INST] Context: May 24 work progress is recorded. Question: What are the concentrations of two PCR tubes after cscBKA fragment amplification? [/INST] About 145 ng/µL for both tubes."}
{"text": "[INST] Context: June 18 work progress is recorded. Question: Which two plasmids arrived from addgene on June 18? [/INST] pRL623 and pRL443 arrived on June 18."}
{"text": "[INST] Context: June 27 experiment issue is recorded. Question: What problem occurred with new Amp plates on June 27? [/INST] New Amp plates were pale, watery after plating, bacteria grew but not well."}
{"text": "[INST] Context: July 2 plasmid extraction results are recorded. Question: What are the OD260/OD280 and OD260/OD230 ratios for sample B-3? [/INST] OD260/OD280: 1.97; OD260/OD230: 1.79 for B-3."}
{"text": "[INST] Context: July 3 work results are recorded. Question: What was the sequencing result for pOpen-cscBKA on July 3? [/INST] Sequencing failed to match, likely abnormal, resequencing required."}
{"text": "[INST] Context: Lead (Pb) remediation enzyme is described. Question: What enzyme is proposed for Pb and its mechanism? [/INST] PbrD secretes phosphatase to precipitate Pb²⁺ as lead phosphate."}
{"text": "[INST] Context: Chromium (Cr) remediation enzyme is described. Question: What enzyme is proposed for Cr and its mechanism? [/INST] ChrA reduces toxic Cr⁶⁺ to less toxic Cr³⁺."}
{"text": "[INST] Context: Strategy to improve enzyme stability is mentioned. Question: How can enzyme stability be improved? [/INST] Replace sensitive amino acids with rare amino acids to improve stability."}
{"text": "[INST] Context: ExPASy PeptideCutter tool is used. Question: What is the purpose of ExPASy PeptideCutter? [/INST] Predict protease-sensitive sites in proteins."}
{"text": "[INST] Context: Production termination feedback loop is designed. Question: How does the negative feedback loop work? [/INST] When product concentration rises, it binds repressor, which blocks RNA polymerase, reducing gene expression."}
{"text": "[INST] Context: Device design is inspired by nature. Question: What marine organism inspired the filter design? [/INST] Filter design is inspired by manta ray feeding mechanism."}
{"text": "[INST] Context: pbrD gene amplification is described. Question: What is the forward primer name and sequence for pbrD amplification? [/INST] Forward primer: PbrDpET32Xa/LICF1; sequence: 5'-GGTATTGAGGGTCGCTTGGCAATTGAG-3'."}
{"text": "[INST] Context: pbrD PCR conditions are described. Question: What is the annealing temperature and cycle number for pbrD PCR? [/INST] Annealing temperature: 62°C; cycles: 35."}
{"text": "[INST] Context: rPbrD overexpression is described. Question: What inducer and final concentration are used for rPbrD overexpression? [/INST] IPTG at 1mM is used for induction."}
{"text": "[INST] Context: rPbrD purification is described. Question: What is the imidazole concentration in elution buffer for Ni-NTA purification? [/INST] Imidazole: 400mM in elution buffer."}
{"text": "[INST] Context: rPbrD refolding is described. Question: What are the final concentrations of urea and DTT in dialysis buffer? [/INST] Urea: 1M; DTT: 1mM in dialysis buffer."}
{"text": "[INST] Context: pbrD gene source is described. Question: What is the source strain and GenBank accession for pbrD? [/INST] Cupriavidus metallidurans CH34; GenBank: CP000354.2."}
{"text": "[INST] Context: Bradford protein assay is mentioned. Question: What is the principle of Bradford assay? [/INST] Coomassie Brilliant Blue binds aromatic and basic amino acids, shifts to blue, absorbance at 595nm."}
{"text": "[INST] Context: Submarine device design includes multiple layers. Question: After fluorescence signal in detection layer, what does production layer produce? [/INST] Production layer produces corresponding aTF after fluorescence signal."}
{"text": "[INST] Context: Light-controlled logic gates regulate aTF production. Question: Which systems respond to red and blue light? [/INST] Red: PhyB/PIF system; Blue: unnamed system (ZPTG in diagram)."}
{"text": "[INST] Context: AraC protein and small molecules are involved in red light pathway. Question: Which small molecule induces aTF1 expression via AraC? [/INST] Arabinose (ara) induces aTF1 via AraC."}
{"text": "[INST] Context: MalT protein and small molecules are involved in blue light pathway. Question: Which small molecule induces aTF2 expression via MalT? [/INST] Maltose (mal) induces aTF2 via MalT."}
{"text": "[INST] Context: aTF secretion test is designed. Question: Which protein is expressed in E. coli for secretion validation? [/INST] TetR protein is expressed to test secretion."}
{"text": "[INST] Context: Promoter strength test is designed. Question: Which fluorescent protein is used as reporter? [/INST] GFP is used as reporter for promoter strength test."}
{"text": "[INST] Context: Light intensity gradient test is designed. Question: What is the range of light intensity for GFP expression test? [/INST] 10 to 100 µmol/m²/s light intensity is used."}
{"text": "[INST] Context: rNTP recovery efficiency test is designed. Question: What method is used to detect rNTP purity and concentration? [/INST] HPLC with C18 column and 260nm UV detection is used."}
{"text": "[INST] Context: E. coli HB101 strain is used. Question: What plasmid backbone does HB101 carry? [/INST] HB101 carries pBR322 plasmid backbone."}
{"text": "[INST] Context: pET series plasmids are used. Question: What method is used to connect cscBKA fragment to linearized pET plasmid? [/INST] Gibson assembly is used for connection."}
{"text": "[INST] Context: May 19 experiment failure is recorded. Question: Why did DH5a not grow after plasmid transformation? [/INST] Possible reasons: failed transformation or inactive DH5a strain."}
{"text": "[INST] Context: June 27 Ampicillin plate issue is recorded. Question: What problem and possible cause for new Amp plates? [/INST] Plates were pale, watery, poor growth; likely not dried before plating."}
{"text": "[INST] Context: July 13 colony PCR results are recorded. Question: How long and at what voltage was colony PCR run, and what minor issue occurred? [/INST] 40 min at 125V; bands well separated but slightly overrun."}
{"text": "[INST] Context: BG11 medium pH adjustment is described. Question: If target pH differs by ~1, how much acid/base to add each time? [/INST] Add 10µL acid/base, stir 1 min, then measure."}
{"text": "[INST] Context: Helper plasmid pRL623 is used. Question: What genes/fragments does pRL623 carry and their function? [/INST] pRL623 carries AvaiM, Eco47iiM, Ecot22iM methylase genes to protect transferred DNA from restriction enzymes."}
{"text": "[INST] Context: Helper plasmid pRL443 is used. Question: What is the antibiotic resistance and concentration for pRL443? [/INST] Ampicillin resistance; 100 µg/mL concentration."}
{"text": "[INST] Context: June 20 single colony isolation is described. Question: What method was used to isolate single colonies from dense culture? [/INST] Dilute in 10µL ddH₂O, streak 1µL to isolate single colonies."}
{"text": "[INST] Context: Plasmid extraction issues are recorded. Question: What common problem is mentioned for June 20 and 28? [/INST] Plasmid concentration consistently low regardless of kit used."}
{"text": "[INST] Context: Various E. coli strains are used. Question: What is the genotype of BL21(DE3)? [/INST] BL21(DE3) genotype: F- ompT gal dcm lon hsdS(rB- mB-) (DE3)."}
{"text": "[INST] Context: Protein nanoparticles as scaffolds are mentioned. Question: What valencies can protein nanoparticles have as scaffolds? [/INST] Valency can be 12, 24, 60, or 180."}
{"text": "[INST] Context: Dodecin-monomer self-assembles into dodecamer. Question: Which termini can SpyTag or SnpTag be attached to in Dodecin-monomer? [/INST] SpyTag/SnpTag can be attached to C- and N-termini."}
{"text": "[INST] Context: pbrD gene is used. Question: What is the full nucleotide sequence of pbrD? [/INST] pbrD nucleotide sequence: CTACCTACAGGCGTAGGCACCGTCGTTGCCCCAACGCCGCCTCATCGATCGCGCGCGCCAAAGCTCGTGTCGGAACCCATTGGCCCCCTTGCGCAATGAATCGCGCGGACGCGTCAACGACGGGTTGCTTGCTCTCATCCACGGATGCGGCGAAGACGGGGGCAACGACCGACTCCGCGAGCGCAAATTGCTGCTTTTCGAATGCCCCTGGATCGAGGCAACCTTCGGCATCGAACGTGAGAATGGAAAACGTCGTTCGAACGACGCGCACTGGCTCTCTGGCCACCAGTTCGACGACCACATCGGCCATGCGTGCGCGTTGCCCGGCAAAAGTGGACAAGCAGTGGTTGGCCGGCTCCCGCAACATCCGCTGGAATTTGGTGGTTGGCAGCTGCCAGAGAGTATCGTCACGGGCAATGAGAAACTTTCGGCAAGAAAACCCCATAGCTGCCCCCATGGACTTTGAGATTCCACAATGTCTCGTCCCGCTCGGACGGTTCTCCCCGCACATCGTACACCGAGAGCATGACCGGGACTTGCGCTGTGACGAAGGTCGCTTCCCGGCCAGAAGCCCCCTGGCTCCCTCGCATGAAAGCCGCCATTCCGTGCCGTTTTCGCCCCCGATGTCGCGCGTCGCCTGCAACGAAATCATGCAAGTTTTGGGTTGTAGGGCGGCGCCTCGCGCCAGACGTCGTTGCAAATCAGCCAAATACACTGGCATCATGGGTGTTCGGCTACTGTCTCTTATGCGGTTGCGACCATGCTCCACGACCAGATCGTTGACATCCTCCTGAGTAGGGCGTCCACTCCCGAACAACTGAGTGCGCCGAGCTGTCCAGGCGTGTATGCATTCTTTCTCAATTGCCAA."}
{"text": "[INST] Context: linker+spyC fusion protein is designed. Question: What is the nucleotide sequence of linker+spyC fusion protein? [/INST] linker+spyC nucleotide sequence: GGTGGCAGCGGTGCTATGGTGGACACCCTGTCGGGCCTCTCTAGTGAACAGGGGCAAAGCGGCGATATGACTATCGAAGAAGATAGTGCTACCCATATTAAATTCTCAAAACGTGATGAGGACGGCAAAGAGTTAGCTGGTGCAACTATGGAGTTGCGTGATTCATCTGGTAAAACTATTAGTACATGGATTTCAGATGGACAAGTGAAAGATTTCTACCTGTATCCAGGAAAATATACATTTGTCGAAACCGCAGCACCAGACGGTTATGAGGTAGCAACTGCTATTACCTTTACAGTTAATGAGCAAGGTCAGGTTACTGTAAATGGCAAAGCAACTAAAGGTGACGCTCATATTCTCGAG TGA."}
{"text": "[INST] Context: Colony PCR reaction system is described. Question: What are the recommended volumes for 2X Taq mix, F-primer, R-primer, and culture in a 10µL colony PCR? [/INST] 2X Taq mix: 5µL; F-primer: 0.3µL; R-primer: 0.3µL; culture: 1-2µL."}
{"text": "[INST] Context: SDS-PAGE results are mentioned. Question: At what molecular weight (kDa) is the mtDod-SpyC-H8 band on SDS-PAGE? [/INST] mtDod-SpyC-H8 band is at ~37 kDa."}
{"text": "[INST] Context: SDS-PAGE results are mentioned. Question: After mClover3-SnpC reacts with SnpT-mtDod-SpyT, at what molecular weight (kDa) is the double conjugate band? [/INST] Double conjugate band is at ~75 kDa."}
{"text": "[INST] Context: Submarine device design includes multiple layers. Question: What signals are transmitted between detection and production layers? [/INST] Detection layer sends light signal to production layer; production layer sends aTF supplement to detection layer."}
{"text": "[INST] Context: Production layer light-controlled logic gates are designed. Question: In the logic gate diagram, which protein's expression is repressed by Trp? [/INST] Trp represses MalT expression in the logic gate diagram."}
{"text": "[INST] Context: Production layer light-controlled logic gates are designed. Question: In the logic gate diagram, which promoter is inhibited by his? [/INST] his inhibits AraC promoter in the logic gate diagram."}
{"text": "[INST] Context: Various E. coli strains are used. Question: What is the genotype of HB101? [/INST] HB101 genotype: supE44, Δ(mcrC-mrr), recA13, ara-14, proA2, lacY1, galK2, rpsL20, xyl-5, mtl-1, leuB6, thi-1."}
{"text": "[INST] Context: Plasmid copy number is mentioned. Question: Is pRL443 high or low copy plasmid? [/INST] pRL443 is a low copy plasmid."}
{"text": "[INST] Context: June 29 work plan is recorded. Question: Which two sequencing primers are designed on June 29? [/INST] pOpen-cscBKA-seq and pBR322-cscB_3-T-NS3 primers are designed."}
{"text": "[INST] Context: July 1 PCR conditions are recorded. Question: What is the extension time for 1-NSC-P fragment PCR on July 1? [/INST] Extension time: 15 seconds for 1-NSC-P PCR."}
{"text": "[INST] Context: July 2 kit issue is recorded. Question: What problem was found with EasyExtract kits on July 2? [/INST] EasyExtract kits gave low plasmid concentration (30-40 ng/µL), below normal."}
{"text": "[INST] Context: July 2 BG11 medium pH is measured. Question: What is the measured pH of BG11 on July 2? [/INST] pH: 6.74 for BG11 on July 2."}
{"text": "[INST] Context: Hydrocarbon-degrading enzymes are designed. Question: Which two hydroxylases degrade C8-C36 alkanes? [/INST] AlkB and P450 hydroxylases degrade C8-C36 alkanes."}
{"text": "[INST] Context: Various plasmids are used. Question: What is the resistance and concentration for pRL623? [/INST] Chloramphenicol resistance; 25 µg/mL for pRL623."}
{"text": "[INST] Context: E. coli culture conditions in symbiosis are specified. Question: What is the shaker speed for E. coli monoculture? [/INST] 200 rpm for E. coli monoculture."}
{"text": "[INST] Context: Cyanobacteria culture conditions in symbiosis are specified. Question: What is the shaker speed for cyanobacteria monoculture? [/INST] 130 rpm for cyanobacteria monoculture."}
{"text": "[INST] Context: Various plasmids are used. Question: What resistance genes are on pACYC184? [/INST] pACYC184 carries chloramphenicol (CmR) and tetracycline (TcR) resistance genes."}
{"text": "[INST] Context: Various strains are used. Question: What is the name of engineered E. coli that utilizes sucrose in symbiosis? [/INST] Engineered E. coli is named cscN or ABKm."}
{"text": "[INST] Context: Various strains are used. Question: What is the name of engineered cyanobacteria that secretes sucrose in symbiosis? [/INST] Engineered cyanobacteria is named cscB 2973."}
{"text": "[INST] Context: June 22 work progress is recorded. Question: Which plasmids were sequenced and verified on June 22? [/INST] pRL443 and pRL623 were sequenced and verified on June 22."}
{"text": "[INST] Context: June 25 work progress is recorded. Question: Which plasmids were transformed on June 25 and for what purpose? [/INST] pBR322 and pOpen plasmids were transformed for expansion and PCR fragment preparation."}
{"text": "[INST] Context: June 30 work progress is recorded. Question: Which construct was sent for sequencing on June 30? [/INST] pOpen-cscBKA construct was sent for sequencing on June 30."}
{"text": "[INST] Context: Various reagents and strains need to be purchased. Question: Which company was HB101 strain purchased from? [/INST] HB101 was purchased from Takara Bio."}
{"text": "[INST] Context: Various reagents and strains need to be purchased. Question: Which company were DH5α and BL21(DE3) strains purchased from? [/INST] DH5α and BL21(DE3) were purchased from TransGen Biotech."}
{"text": "[INST] Context: Various PCR enzymes are used. Question: What is the product number for Vazyme 2x Phanta Flash Master Mix (Dye Plus)? [/INST] Product number: P520 for Vazyme 2x Phanta Flash Master Mix (Dye Plus)."}
{"text": "[INST] Context: Various DNA Markers are used. Question: What is the product number for Takara DL2,000 DNA Marker? [/INST] Product number: 3427A for Takara DL2,000 DNA Marker."}
{"text": "[INST] Context: Copper (Cu) remediation enzymes are designed. Question: What enzymes are proposed for Cu remediation? [/INST] CneO and CusF are proposed for Cu remediation."}
{"text": "[INST] Context: Nickel (Ni) remediation enzymes are designed. Question: What proteins are proposed for Ni remediation? [/INST] CnrH, CnrX, and CnrY are proposed for Ni remediation."}
{"text": "[INST] Context: Plasmid pJA is mentioned. Question: What resistance genes are on pJA? [/INST] pJA carries spectinomycin (speR) and kanamycin (kanR) resistance genes."}
{"text": "[INST] Context: July 14 BG11 medium pH is recorded. Question: What is the final pH of BG11 prepared on July 14? [/INST] Final pH: 7.31 for BG11 on July 14."}
"""





lora_dataset_path = "./my_lora_dataset.jsonl"

try:
    with open(lora_dataset_path, "w", encoding="utf-8") as f:
        f.write(lora_dataset_content.strip())
    print(f"Dataset file '{lora_dataset_path}' created successfully")
except IOError as e:
    print(f"Error creating file: {e}")




In [ ]:
import torch
from datasets import load_dataset, Dataset
import json
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
import os
from tqdm import tqdm
base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
lora_dataset_path = "./my_lora_dataset.jsonl"
output_dir = "./results"
new_model_path = "./Llama-3-8B-custom-RAG-zh"

main_steps = ["Load dataset", "Load model and tokenizer", "Configure and start training", "Save adapter"]

with tqdm(total=len(main_steps), desc="Overall fine-tuning progress") as main_pbar:

    main_pbar.set_description(f"Step: {main_steps[0]}")
    if not os.path.exists(lora_dataset_path):
        raise FileNotFoundError(
            f"Error: Dataset file '{lora_dataset_path}' not found. "
            f"Please run prepare_dataset.py first to generate this file."
        )

    data = []
    with open(lora_dataset_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))

    dataset = Dataset.from_list(data)

    main_pbar.update(1)

    main_pbar.set_description(f"Step: {main_steps[1]}")
    bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
    model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto")
    model.config.use_cache = False
    tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"
    main_pbar.update(1)

    main_pbar.set_description(f"Step: {main_steps[2]}")
    lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"])
    training_arguments = TrainingArguments(output_dir=output_dir, num_train_epochs=10, per_device_train_batch_size=2, gradient_accumulation_steps=4, optim="paged_adamw_32bit", logging_steps=10, learning_rate=2e-4, bf16=True, max_grad_norm=0.3, warmup_ratio=0.03, group_by_length=True, report_to='none')

    trainer = SFTTrainer(
        model=model,
        train_dataset=dataset,
        peft_config=lora_config,
        args=training_arguments,
    )

    trainer.train()
    main_pbar.update(1)

    main_pbar.set_description(f"Step: {main_steps[3]}")
    os.makedirs(new_model_path, exist_ok=True)
    trainer.model.save_pretrained(new_model_path)
    tokenizer.save_pretrained(new_model_path)
    main_pbar.update(1)

print("\nAll steps completed successfully through Google Colab, for any issues please contact the Dry Lab of ZJU-China 2025.")
print("Please do not use this model for any unauthorized purposes. All rights reserved.")
